- Прочитайте главы 4-6 из книги "Spark: The Definitive Guide".
- Загрузите датасет из предыдущей лабораторной работы:
    - https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2022
- Выполните задания.

In [1]:
# Создаем SparkSession
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkLab2') \
                    .getOrCreate()

# Загружаем датасет.
tiktokData0 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/home/student/labs-1/TikTok_songs_2022.csv")

22/09/16 16:29:33 WARN Utils: Your hostname, student-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/09/16 16:29:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/16 16:29:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/16 16:29:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/16 16:29:36 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


№1: В столбце "loudness" переведите значения из дБ в проценты громкости. Выведите топ10 самых громких песен.

|          track_name|               album|        artist_name|loudness|
|--------------------|--------------------|-------------------|--------|
|          Astronomia|          Astronomia|           Vicetone|      55|
|     Sweater Weather|         I Love You.|  The Neighbourhood|      52|
|          Dandelions|          Safe Haven|            Ruth B.|      51|
|1, 2, 3 (feat. Ja...|1, 2, 3 (feat. Ja...|        Sofía Reyes|      49|
|   Beauty And A Beat|   Beauty And A Beat|              Other|      49|
|Being Good Is Boring|Being Good Is Boring|              Other|      48|
|Wellerman - Sea S...|Wellerman (Sea Sh...|       Nathan Evans|      47|
|           Thot Shit|           Thot Shit|Megan Thee Stallion|      45|
|Friday (feat. Muf...|Friday (feat. Muf...|              Riton|      45|
|               Hawái|        PAPI JUANCHO|             Maluma|      45|

In [250]:
from pyspark.sql.functions import desc
from pyspark.sql.functions import col
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType

tiktokData1 = tiktokData0\

tiktokData1=tiktokData1.sort(desc("loudness"))\
.select("track_name", "album", "artist_name", "loudness")\
.withColumn("loudness", round(100*10**(col("loudness")/10)).cast("int"))\
.show(100)

+--------------------+--------------------+--------------------+--------+
|          track_name|               album|         artist_name|loudness|
+--------------------+--------------------+--------------------+--------+
|          Astronomia|          Astronomia|            Vicetone|      55|
|     Sweater Weather|         I Love You.|   The Neighbourhood|      52|
|          Dandelions|          Safe Haven|             Ruth B.|      51|
|1, 2, 3 (feat. Ja...|1, 2, 3 (feat. Ja...|         Sofía Reyes|      49|
|   Beauty And A Beat|   Beauty And A Beat|              Glamii|      49|
|Being Good Is Boring|Being Good Is Boring|           Jena Rose|      48|
|Wellerman - Sea S...|Wellerman (Sea Sh...|        Nathan Evans|      47|
|           Thot Shit|           Thot Shit| Megan Thee Stallion|      45|
|Friday (feat. Muf...|Friday (feat. Muf...|               Riton|      45|
|      Break My Heart|    Future Nostalgia|            Dua Lipa|      45|
|               Hawái|        PAPI JUA

№2: Получите имена соисполнителей из названий песен. Вынесите их в отдельный столбец. Удалите информацию о соисполнителях из названий треков.

|track_name                                     |artist_name      |feat                       |
|-----------------------------------------------|-----------------|---------------------------|
|INDUSTRY BABY                                  |Lil Nas X        |Jack Harlow                |
|Left and Right (Charlie Puth) - Sped Up Version|sped up nightcore|Jung Kook of BTS           |
|Bam Bam                                        |Camila Cabello   |Ed Sheeran                 |
|Down Under                                     |Luude            |Colin Hay                  |
|1, 2, 3                                        |Sofía Reyes      |Jason Derulo & De La Ghetto|
|Chaa Chaa Chaa                                 |Girll Codee      |HoodCelebrityy             |

<div style="text-align: center"> only showing top 6 rows </div>

In [290]:
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import regexp_replace

tiktokData1 = tiktokData0\

d1 = tiktokData1.withColumn("feat", regexp_extract(col("track_name"), "[\(\[]feat. ([^\)]*)[\)\]]",1))\
.select("album","track_name", "artist_name", "feat", "artist_pop")


d1.select("*").where(col("track_name").ilike ("%feat%")).show(1000)

+--------------------+--------------------+-----------------+--------------------+----------+
|               album|          track_name|      artist_name|                feat|artist_pop|
+--------------------+--------------------+-----------------+--------------------+----------+
|INDUSTRY BABY (fe...|INDUSTRY BABY (fe...|        Lil Nas X|         Jack Harlow|        81|
|Left and Right (C...|Left and Right (C...|sped up nightcore|    Jung Kook of BTS|        63|
|             Familia|Bam Bam (feat. Ed...|   Camila Cabello|          Ed Sheeran|        82|
|Down Under (feat....|Down Under (feat....|            Luude|           Colin Hay|        68|
|1, 2, 3 (feat. Ja...|1, 2, 3 (feat. Ja...|      Sofía Reyes|Jason Derulo & De...|        72|
|Chaa Chaa Chaa (f...|Chaa Chaa Chaa (f...|      Girll Codee|      HoodCelebrityy|        29|
|Running To You (f...|Running To You (f...|            VINAI|               Caden|        64|
|SAD GIRLZ LUV MON...|SAD GIRLZ LUV MON...|          Amaarae

№3: Разделите строки с соисполнителями по символу '&'. Для каждой песни соберите всех исполнителей в один массив. Отсортируйте песни по количеству исполнителей и названиям песен.

|track_name                       |artist_names                             |
|---------------------------------|-----------------------------------------|
|1, 2, 3                          |[Sofía Reyes, Jason Derulo, De La Ghetto]|
|For The Night                    |[Pop Smoke, Lil Baby, DaBaby]            |
|Friday - Dopamine Re-Edit        |[Riton, Mufasa, Hypeman]                 |
|Peaches                          |[Justin Bieber, Daniel Caesar, Giveon]   |
|Bam Bam                          |[Camila Cabello, Ed Sheeran]             |
|Banana - DJ FLe - Minisiren Remix|[Conkarah, Shaggy]                       |

<div style="text-align: center"> only showing top 6 rows </div>

In [350]:
from pyspark.sql.functions import split
from pyspark.sql.functions import array_union
from pyspark.sql.functions import array
from pyspark.sql.functions import size
from pyspark.sql.functions import when

tiktokData1 = tiktokData0\

d2=d1.select("artist_pop", "album","track_name","artist_name",split("artist_name"," & ").alias("art"), split("feat"," & ").alias("name")) \
.select("artist_pop", "album","track_name",array_union(col("art"),col("name")).alias("artist"))\
.sort(desc(size("artist")), "track_name")



d2.select("track_name","artist").show(100,False)



+--------------------------------------------------------------+-----------------------------------------+
|track_name                                                    |artist                                   |
+--------------------------------------------------------------+-----------------------------------------+
|1, 2, 3 (feat. Jason Derulo & De La Ghetto)                   |[Sofía Reyes, Jason Derulo, De La Ghetto]|
|Don't Rush (feat. Headie One)                                 |[Young T, Bugsey, Headie One]            |
|For The Night (feat. Lil Baby & DaBaby)                       |[Pop Smoke, Lil Baby, DaBaby]            |
|Friday (feat. Mufasa & Hypeman) - Dopamine Re-Edit            |[Riton, Mufasa, Hypeman]                 |
|Jiggle Jiggle                                                 |[Duke, Jones, ]                          |
|Looking for Love                                              |[Asketa, Natan Chaim, ]                  |
|Peaches (feat. Daniel Caesar & Giveo

№4: Выведите список песен Doja Cat.

|  track_name|               album|        artist_names|
|------------|--------------------|--------------------|
|       Woman|          Planet Her|          [Doja Cat]|
|Kiss Me More|Kiss Me More (fea...|     [Doja Cat, SZA]|
|Need to Know|          Planet Her|          [Doja Cat]|
|  Ain't Shit|          Planet Her|          [Doja Cat]|
|   You Right|          Planet Her|          [Doja Cat]|
| Best Friend|Best Friend (feat...|[Saweetie, Doja Cat]|
|       Freak|               Freak|          [Doja Cat]|
|  Boss Bitch|          Boss Bitch|          [Doja Cat]|
|      Say So|            Hot Pink|          [Doja Cat]|

In [274]:
from pyspark.sql.functions import explode

d2.select("track_name","album","artist",explode("artist").alias("art_ch"))\
.where(col("art_ch").ilike("%Doja Cat%"))\
.show(2000)




+--------------------+--------------------+--------------------+--------+
|          track_name|               album|              artist|  art_ch|
+--------------------+--------------------+--------------------+--------+
|          Ain't Shit|          Planet Her|        [Doja Cat, ]|Doja Cat|
|Best Friend (feat...|Best Friend (feat...|[Saweetie, Doja Cat]|Doja Cat|
|          Boss Bitch|          Boss Bitch|        [Doja Cat, ]|Doja Cat|
|               Freak|               Freak|        [Doja Cat, ]|Doja Cat|
|Kiss Me More (fea...|Kiss Me More (fea...|     [Doja Cat, SZA]|Doja Cat|
|        Need to Know|          Planet Her|        [Doja Cat, ]|Doja Cat|
|              Say So|            Hot Pink|        [Doja Cat, ]|Doja Cat|
|               Woman|          Planet Her|        [Doja Cat, ]|Doja Cat|
|           You Right|          Planet Her|        [Doja Cat, ]|Doja Cat|
+--------------------+--------------------+--------------------+--------+



№5: Выведите таблицу с исполнителями и количеством их треков в порядке уменьшения.

|   artist_name|count|
|--------------|-----|
|      Doja Cat|    9|
|        Coopex|    6|
|     Dame Dame|    5|
|Alex Alexander|    4|
|       YES YES|    4|
|     Lil Nas X|    4|


<div style="text-align: center"> only showing top 6 rows </div>

In [279]:
tiktokData1 = tiktokData0\

df = d2.select("track_name","album","artist",explode("artist").alias("art_ch"))

df.select("art_ch")\
    .groupBy("art_ch")\
    .count()\
    .where(col("art_ch") != '')\
    .sort(desc("count"))\
    .show(10, False)


+-------------------+-----+
|art_ch             |count|
+-------------------+-----+
|Doja Cat           |9    |
|Coopex             |6    |
|Dame Dame          |5    |
|Alex Alexander     |4    |
|Lil Nas X          |4    |
|Drake              |4    |
|YES YES            |4    |
|Megan Thee Stallion|4    |
|Farux              |4    |
|DaBaby             |4    |
+-------------------+-----+
only showing top 10 rows



№6: Замените имена всех исполнителей, популярность которых неизвестна или меньше 50, на "Other". Отсортируйте треки по количеству исполнителей и популярности.

|track_name               |artist_names                   |track_pop|
|-------------------------|-------------------------------|---------|
|Peaches                  |[Justin Bieber, Other, Other]  |86       |
|Friday - Dopamine Re-Edit|[Riton, Other, Other]          |83       |
|1, 2, 3                  |[Sofía Reyes, Other, Other]    |82       |
|For The Night            |[Pop Smoke, Other, DaBaby]     |80       |
|Jimmy Cooks              |[Drake, Other]                 |92       |
|INDUSTRY BABY            |[Lil Nas X, Jack Harlow]       |86       |
|Levitating               |[Dua Lipa, DaBaby]             |85       |
|Bam Bam                  |[Camila Cabello, Ed Sheeran]   |83       |
|Beautiful Mistakes       |[Maroon 5, Megan Thee Stallion]|82       |
|Kiss Me More             |[Doja Cat, SZA]                |82       |

<div style="text-align: center"> only showing top 10 rows </div>

In [369]:
from pyspark.sql.functions import collect_list

tiktokData1 = tiktokData0\

ff = d2.select("track_name", "artist", "artist_pop", explode("artist").alias("artist_name"))

D = d1.select("track_name", "artist_name", "artist_pop")\
    .where(col("artist_pop")>50)

states1=D.rdd.map(lambda x: x[1]).collect()

dg = ff.select("track_name", "artist_name", "artist_pop", when(ff.artist_name.isin(states1), ff.artist_name)\
    .otherwise("other").alias("new"))

dg = dg.groupBy("track_name").agg(collect_list("new").alias("artist_names"))

dg.select("*").show(300, False)

+------------------------------------------------------------------------+--------------------------------------------+
|track_name                                                              |artist_names                                |
+------------------------------------------------------------------------+--------------------------------------------+
|"More Than A Woman - From ""Saturday Night Fever"" Soundtrack"          |[Bee Gees, other]                           |
|1, 2, 3 (feat. Jason Derulo & De La Ghetto)                             |[Sofía Reyes, other, other]                 |
|34+35                                                                   |[Ariana Grande, other]                      |
|AMG                                                                     |[other, other]                              |
|About Damn Time                                                         |[Lizzo, other]                              |
|Aesthetic                              